In [5]:
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000.00,
        'Loans Outstanding': 2000000.00,
        'Cash Reserves': 1000000.00,
        'NOSTRO Account': 500000.00,
        'Fee Income': 0.00,
        'Interest Income': 0.00
    }

# Helper function to calculate 10% transaction fee
def apply_fee(amount):
    return round(amount * 0.10, 2)

# Helper function to add transaction to the ledger
def add_to_ledger(transaction_ledger, date, transaction_type, dr_cr, amount, account, balance_after, sequence_num):
    transaction_ledger.append([date, sequence_num, transaction_type, dr_cr, amount, account, balance_after])

# Function to check if the debits and credits match for a transaction
def check_balance(transaction_ledger):
    total_debits = sum(entry[4] for entry in transaction_ledger if entry[3] == 'Debit')
    total_credits = sum(entry[4] for entry in transaction_ledger if entry[3] == 'Credit')
    if round(total_debits, 2) != round(total_credits, 2):
        # If they don't match, print the transaction ledger for debugging purposes
        print(f"Unbalanced transaction:")
        for entry in transaction_ledger:
            print(entry)
        raise ValueError(f"Unbalanced transaction: Debits = {total_debits}, Credits = {total_credits}")

# Function to process transactions sequentially and ensure balance
def process_transaction(balances, daily_ledger, transaction_type, main_amount, sequence_num, date):
    transaction_ledger = []
    if transaction_type == 'Deposit':
        deposit_fee = apply_fee(main_amount)
        net_deposit = main_amount - deposit_fee

        # Update balances
        balances['Customer Deposits'] += net_deposit
        balances['Cash Reserves'] += net_deposit
        balances['Fee Income'] += deposit_fee

        # Record the transactions
        add_to_ledger(transaction_ledger, date, 'Deposit', 'Debit', net_deposit, 'Cash Reserves', balances['Cash Reserves'], sequence_num)
        add_to_ledger(transaction_ledger, date, 'Deposit', 'Credit', net_deposit, 'Customer Deposits', balances['Customer Deposits'], sequence_num)
        add_to_ledger(transaction_ledger, date, 'Deposit Fee', 'Credit', deposit_fee, 'Fee Income', balances['Fee Income'], sequence_num)

    elif transaction_type == 'Loan':
        loan_fee = apply_fee(main_amount)
        net_loan_disbursed = main_amount - loan_fee

        if balances['Cash Reserves'] >= net_loan_disbursed:
            balances['Loans Outstanding'] += net_loan_disbursed
            balances['Cash Reserves'] -= net_loan_disbursed
            balances['Fee Income'] += loan_fee

            # Record the transactions
            add_to_ledger(transaction_ledger, date, 'Loan Disbursed', 'Debit', net_loan_disbursed, 'Loans Outstanding', balances['Loans Outstanding'], sequence_num)
            add_to_ledger(transaction_ledger, date, 'Loan Disbursed', 'Credit', net_loan_disbursed, 'Cash Reserves', balances['Cash Reserves'], sequence_num)
            add_to_ledger(transaction_ledger, date, 'Loan Fee', 'Credit', loan_fee, 'Fee Income', balances['Fee Income'], sequence_num)
        else:
            print("Not enough Cash Reserves to disburse loan.")
            return  # Skip this transaction if not enough cash reserves

    elif transaction_type == 'Client Payment':
        if balances['Customer Deposits'] >= main_amount:
            balances['Customer Deposits'] -= main_amount
            balances['Cash Reserves'] -= main_amount

            # Record the transactions
            add_to_ledger(transaction_ledger, date, 'Client Payment', 'Debit', main_amount, 'Customer Deposits', balances['Customer Deposits'], sequence_num)
            add_to_ledger(transaction_ledger, date, 'Client Payment', 'Credit', main_amount, 'Cash Reserves', balances['Cash Reserves'], sequence_num)
        else:
            print("Not enough Customer Deposits for client payment.")
            return  # Skip this transaction if not enough customer deposits

    elif transaction_type == 'Treasury Transaction':
        if balances['NOSTRO Account'] >= main_amount:
            balances['NOSTRO Account'] -= main_amount
            balances['Cash Reserves'] += main_amount

            # Record the transactions
            add_to_ledger(transaction_ledger, date, 'Treasury Transaction', 'Debit', main_amount, 'Cash Reserves', balances['Cash Reserves'], sequence_num)
            add_to_ledger(transaction_ledger, date, 'Treasury Transaction', 'Credit', main_amount, 'NOSTRO Account', balances['NOSTRO Account'], sequence_num)
        else:
            print("Not enough funds in NOSTRO Account for treasury transaction.")
            return  # Skip this transaction if not enough NOSTRO funds

    elif transaction_type == 'Interest Income':
        balances['Cash Reserves'] += main_amount
        balances['Interest Income'] += main_amount

        # Record the transactions
        add_to_ledger(transaction_ledger, date, 'Interest Income', 'Debit', main_amount, 'Cash Reserves', balances['Cash Reserves'], sequence_num)
        add_to_ledger(transaction_ledger, date, 'Interest Income', 'Credit', main_amount, 'Interest Income', balances['Interest Income'], sequence_num)

    # Check if the transaction ledger is balanced
    check_balance(transaction_ledger)

    # If balanced, append transaction ledger to daily ledger
    daily_ledger.extend(transaction_ledger)

# Simulate daily transactions one by one, ensuring balance for each transaction
def simulate_day(balances, sequence_num, date):
    # Initialize daily ledger
    daily_ledger = []

    # Generate random transaction amounts
    transactions = [
        ('Deposit', random.randint(10000, 100000)),
        ('Loan', random.randint(50000, 250000)),
        ('Client Payment', random.randint(5000, 30000)),
        ('Treasury Transaction', random.randint(50000, 150000)),
        ('Interest Income', random.randint(5000, 15000))
    ]

    # Process each transaction sequentially
    for i, (transaction_type, amount) in enumerate(transactions):
        process_transaction(balances, daily_ledger, transaction_type, amount, sequence_num + i, date)

    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    total_assets = balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
    total_liabilities = balances['Customer Deposits']
    total_equity = balances['Fee Income'] + balances['Interest Income']
    if round(total_assets, 2) != round(total_liabilities + total_equity, 2):
        print(f"Unbalanced financial report: Assets = {total_assets}, Liabilities + Equity = {total_liabilities + total_equity}")
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': total_assets
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': total_liabilities
        },
        'Equity': {
            'Fee Income': balances['Fee Income'],
            'Interest Income': balances['Interest Income'],
            'Total Equity': total_equity
        }
    }
    return balance_sheet

# Save the ledger and financial report to SQLite database and CSV files
def save_data_to_db_and_csv(ledger, reports, db_name="bank_ledger.db", ledger_csv="bank_ledger.csv", report_csv="financial_report.csv"):
    # Save the ledger
    df_ledger = pd.DataFrame(ledger, columns=['Date', 'Sequence', 'Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df_ledger.to_sql('ledger', conn, if_exists='replace', index=False)
    df_ledger.to_csv(ledger_csv, index=False)

    # Save the financial reports
    df_reports = pd.DataFrame(reports)
    df_reports.to_csv(report_csv, index=False)

    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    all_reports = []
    sequence_num = 1

    # Start date for the month simulation
    start_date = datetime.now()

    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances, sequence_num, date)
        sequence_num += len(daily_ledger)  # Increment sequence number

        # Append each transaction to all_ledgers
        all_ledgers.extend(daily_ledger)

        # Generate daily financial report
        daily_report = generate_daily_report(balances)
        daily_report['Date'] = date
        all_reports.append(daily_report)

        # Print daily financial report (optional)
        print(f"\n=== Day {day+1} Financial Report for {date} ===")
        df_report = pd.DataFrame(daily_report).T
        print(df_report)

    return all_ledgers, all_reports

# Main execution function
def main():
    balances = initialize_balances()

    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger, monthly_reports = simulate_month(balances)

    # Save the ledger and financial reports to database and CSV files
    print("\nSaving ledger and financial reports to database and CSV files...")
    save_data_to_db_and_csv(monthly_ledger, monthly_reports)
    print("Ledger and financial reports saved successfully!")

if __name__ == '__main__':
    main()



Simulating one month of banking operations...
Unbalanced transaction:
['2024-09-16', 1, 'Deposit', 'Debit', 89833.5, 'Cash Reserves', 1089833.5]
['2024-09-16', 1, 'Deposit', 'Credit', 89833.5, 'Customer Deposits', 5089833.5]
['2024-09-16', 1, 'Deposit Fee', 'Credit', 9981.5, 'Fee Income', 9981.5]


ValueError: Unbalanced transaction: Debits = 89833.5, Credits = 99815.0